# Data

The data set we’ll use is a list of over one million news headlines published over a period of 15 years and can be downloaded from Kaggle (https://www.kaggle.com/therohk/million-headlines/data)

In [1]:
import pandas as pd
import zipfile

zf = zipfile.ZipFile(r"C:\Users\saurabhkumar9\1. NLP Practicum Materials\Data\million-headlines.zip") 
data = pd.read_csv(zf.open('abcnews-date-text.csv'))

In [2]:
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

print(len(documents))
print(documents[:5])

1186018
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


# Data Pre-processing

In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [14]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saurabhkumar9\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**Lemmatize and Stemming**

In [21]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


In [18]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [19]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [20]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')

words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [22]:
processed_docs = documents['headline_text'].map(preprocess) # Applying on all the data

In [23]:
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

# Bag of words on the dataset

In [24]:
dictionary = gensim.corpora.Dictionary(processed_docs)

Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.

In [25]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


**Filter out tokens that appear in**
* less than 15 documents (absolute number) or
* more than 0.5 documents (fraction of total corpus size, not absolute number).
* after the above two steps, keep only the first 100000 most frequent tokens.

In [26]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

**Gensim doc2bow**

For each document we create a dictionary reporting how many words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.

In [27]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (838, 1), (3567, 1), (3568, 1)]

**Preview Bag Of Words for our sample preprocessed document**

In [28]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 162 ("govt") appears 1 time.
Word 240 ("group") appears 1 time.
Word 292 ("vote") appears 1 time.
Word 589 ("local") appears 1 time.
Word 838 ("want") appears 1 time.
Word 3567 ("compulsori") appears 1 time.
Word 3568 ("ratepay") appears 1 time.


# TF-IDF

Create tf-idf model object using models.TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, then apply transformation to the entire corpus and call it ‘corpus_tfidf’. Finally we preview TF-IDF scores for our first document.

In [29]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [30]:
corpus_tfidf = tfidf[bow_corpus]

In [31]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5850076620505259),
 (1, 0.38947256567331934),
 (2, 0.4997099083387053),
 (3, 0.5063271308533074)]


## Running LDA using Bag of Words

Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’

In [32]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

For each topic, we will explore the words occuring in that topic and its relative weight.

In [35]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.022*"hous" + 0.022*"south" + 0.020*"north" + 0.016*"miss" + 0.015*"bushfir" + 0.013*"interview" + 0.012*"west" + 0.011*"hospit" + 0.011*"coast" + 0.011*"investig"
Topic: 1 
Words: 0.031*"kill" + 0.023*"shoot" + 0.021*"protest" + 0.020*"dead" + 0.020*"polic" + 0.020*"attack" + 0.014*"offic" + 0.013*"assault" + 0.013*"chines" + 0.011*"michael"
Topic: 2 
Words: 0.057*"australia" + 0.046*"australian" + 0.026*"world" + 0.018*"canberra" + 0.017*"test" + 0.013*"win" + 0.011*"final" + 0.011*"farm" + 0.010*"return" + 0.010*"beat"
Topic: 3 
Words: 0.030*"polic" + 0.029*"charg" + 0.026*"court" + 0.024*"death" + 0.024*"murder" + 0.020*"woman" + 0.018*"crash" + 0.017*"face" + 0.017*"alleg" + 0.013*"jail"
Topic: 4 
Words: 0.019*"chang" + 0.019*"say" + 0.015*"speak" + 0.015*"power" + 0.013*"worker" + 0.012*"concern" + 0.012*"climat" + 0.011*"polit" + 0.011*"flood" + 0.010*"fear"
Topic: 5 
Words: 0.021*"market" + 0.019*"news" + 0.018*"women" + 0.017*"live" + 0.016*"tasmania" + 0.013

Can you distinguish different topics using the words in each topic and their corresponding weights?

## Running LDA using TF-IDF

In [36]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [ ]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Again, can you distinguish different topics using the words in each topic and their corresponding weights?

# Classification of the topics

## Performance evaluation by classifying sample document using LDA Bag of Words model

We will check where our test document would be classified.

In [37]:
processed_docs[4310]

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

In [38]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.45473378896713257	 
Topic: 0.020*"donald" + 0.014*"nation" + 0.014*"farmer" + 0.013*"rural" + 0.013*"time" + 0.013*"council" + 0.012*"indigen" + 0.012*"school" + 0.011*"plan" + 0.011*"commiss"

Score: 0.4452346861362457	 
Topic: 0.031*"govern" + 0.020*"warn" + 0.019*"feder" + 0.015*"countri" + 0.015*"fund" + 0.015*"claim" + 0.014*"life" + 0.013*"say" + 0.012*"health" + 0.011*"stori"

Score: 0.012506541796028614	 
Topic: 0.036*"elect" + 0.018*"water" + 0.018*"state" + 0.016*"tasmanian" + 0.012*"labor" + 0.012*"liber" + 0.011*"morrison" + 0.011*"parti" + 0.010*"campaign" + 0.010*"give"

Score: 0.012503868900239468	 
Topic: 0.019*"chang" + 0.019*"say" + 0.015*"speak" + 0.015*"power" + 0.013*"worker" + 0.012*"concern" + 0.012*"climat" + 0.011*"polit" + 0.011*"flood" + 0.010*"fear"

Score: 0.012503519654273987	 
Topic: 0.022*"hous" + 0.022*"south" + 0.020*"north" + 0.016*"miss" + 0.015*"bushfir" + 0.013*"interview" + 0.012*"west" + 0.011*"hospit" + 0.011*"coast" + 0.011*"investig"

Our test document has the highest probability to be part of the topic that our model assigned, which is the accurate classification.

## Performance evaluation by classifying sample document using LDA TF-IDF model

In [39]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.44451671838760376	 
Topic: 0.010*"govern" + 0.007*"health" + 0.006*"farm" + 0.006*"queensland" + 0.006*"hill" + 0.006*"fund" + 0.006*"morrison" + 0.005*"council" + 0.005*"budget" + 0.005*"plan"

Score: 0.29844287037849426	 
Topic: 0.025*"trump" + 0.016*"countri" + 0.012*"hour" + 0.011*"stori" + 0.008*"turnbul" + 0.007*"commiss" + 0.007*"royal" + 0.007*"say" + 0.007*"korea" + 0.006*"elect"

Score: 0.1694842129945755	 
Topic: 0.012*"donald" + 0.010*"australia" + 0.009*"world" + 0.007*"market" + 0.007*"australian" + 0.007*"final" + 0.007*"leagu" + 0.007*"street" + 0.007*"weather" + 0.006*"live"

Score: 0.012508596293628216	 
Topic: 0.013*"bushfir" + 0.008*"plead" + 0.008*"violenc" + 0.007*"john" + 0.007*"domest" + 0.007*"child" + 0.007*"abus" + 0.006*"guilti" + 0.006*"asylum" + 0.006*"april"

Score: 0.012508460320532322	 
Topic: 0.026*"news" + 0.021*"rural" + 0.010*"climat" + 0.009*"friday" + 0.008*"nation" + 0.008*"david" + 0.008*"care" + 0.008*"chang" + 0.008*"grandstand" + 0.

Our test document has the highest probability to be part of the topic that our model assigned, which is the accurate classification.

# Testing model on unseen document

In [40]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.48968884348869324	 Topic: 0.021*"market" + 0.019*"news" + 0.018*"women" + 0.017*"live" + 0.016*"tasmania"
Score: 0.21062807738780975	 Topic: 0.036*"elect" + 0.018*"water" + 0.018*"state" + 0.016*"tasmanian" + 0.012*"labor"
Score: 0.1829463094472885	 Topic: 0.030*"polic" + 0.029*"charg" + 0.026*"court" + 0.024*"death" + 0.024*"murder"
Score: 0.016677437350153923	 Topic: 0.019*"chang" + 0.019*"say" + 0.015*"speak" + 0.015*"power" + 0.013*"worker"
Score: 0.016677269712090492	 Topic: 0.031*"govern" + 0.020*"warn" + 0.019*"feder" + 0.015*"countri" + 0.015*"fund"
Score: 0.0166764073073864	 Topic: 0.022*"hous" + 0.022*"south" + 0.020*"north" + 0.016*"miss" + 0.015*"bushfir"
Score: 0.0166764073073864	 Topic: 0.031*"kill" + 0.023*"shoot" + 0.021*"protest" + 0.020*"dead" + 0.020*"polic"
Score: 0.0166764073073864	 Topic: 0.057*"australia" + 0.046*"australian" + 0.026*"world" + 0.018*"canberra" + 0.017*"test"
Score: 0.0166764073073864	 Topic: 0.020*"donald" + 0.014*"nation" + 0.014*"farme